# 🔄 Notebook 2: Flujo Completo con Anonimización

Este notebook ejecuta el flujo completo:
1. Cargar datos
2. Ejecutar análisis en paralelo
3. Detectar PII
4. Anonimizar automáticamente
5. Generar diagnóstico con LLM

## Setup

In [1]:
import pandas as pd
import sys
import json
from pathlib import Path

sys.path.append('..')

from fase3_evaluator.integration import (
    run_parallel_analysis,
    get_analysis_summary
)
from fase3_evaluator.agent import generate_diagnosis
from config import settings

print("✅ Imports exitosos")
print(f"\n⚙️ Configuración:")
print(f"  - OpenAI Model: {settings.OPENAI_MODEL}")
print(f"  - Max Workers: {settings.MAX_WORKERS}")
print(f"  - API Key configurada: {'Sí' if settings.OPENAI_API_KEY else 'No'}")

✅ Imports exitosos

⚙️ Configuración:
  - OpenAI Model: gpt-4o
  - Max Workers: 6
  - API Key configurada: Sí


## Paso 1: Crear Dataset con PII

Creamos un dataset que incluye información sensible para probar la anonimización

In [2]:
# Dataset con PII para probar anonimización
df_with_pii = pd.DataFrame({
    'id_registro': range(1, 16),
    'nombre_completo': [
        'Juan García López', 'María Rodríguez Pérez', 'Pedro Martínez Sánchez',
        'Ana López Fernández', 'Carlos Sánchez García', 'Laura Fernández Martín',
        'Miguel Pérez López', 'Isabel García Rodríguez', 'Antonio López Martínez',
        'Carmen Martínez García', 'José Rodríguez López', 'Teresa García Pérez',
        'Francisco López Sánchez', 'Rosa Pérez Fernández', 'Manuel Sánchez Martín'
    ],
    'fecha_nacimiento': pd.date_range('1960-01-01', periods=15, freq='2Y'),
    'fecha_evento': pd.date_range('2024-01-01', periods=15, freq='2W'),
    'edad': [25, 30, 45, 52, 38, 60, 28, 35, 42, 29, 55, 48, 33, 41, 37],
    'sexo': ['M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M'],
    'dni': ['12345678A', '23456789B', '34567890C', '45678901D', '56789012E',
            '67890123F', '78901234G', '89012345H', '90123456I', '01234567J',
            '12345678K', '23456789L', '34567890M', '45678901N', '56789012O'],
    'telefono': ['666123456', '677234567', '688345678', None, '600456789',
                 '611567890', '622678901', '633789012', None, '655901234',
                 '666012345', '677123456', '688234567', '699345678', '600456789'],
    'direccion': [
        'Calle Mayor 123, Madrid', 'Avenida Principal 45, Valencia',
        'Plaza España 7, Barcelona', 'Calle Real 89, Sevilla',
        'Madrid', 'Valencia', 'Barcelona', 'Calle Nueva 12, Madrid',
        'Sevilla', 'Avenida Central 34, Madrid', 'Valencia',
        'Calle Luna 56, Barcelona', 'Madrid', 'Sevilla', 'Valencia'
    ],
    'metodo': [
        'ahorcamiento', 'intoxicacion', 'arma de fuego', 'ahorcamiento', 'precipitacion',
        'ahorcamiento', 'intoxicacion', None, 'arma de fuego', 'ahorcamiento',
        'intoxicacion', 'ahorcamiento', 'precipitacion', 'intoxicacion', 'ahorcamiento'
    ],
    'municipio': [
        'Madrid', 'Valencia', 'Barcelona', 'Sevilla', 'Madrid',
        'Valencia', 'Barcelona', 'Madrid', 'Sevilla', 'Madrid',
        'Valencia', 'Barcelona', 'Madrid', 'Sevilla', 'Valencia'
    ],
    'tipo_evento': [
        'consumado', 'intento', 'consumado', 'intento', 'consumado',
        'consumado', 'intento', 'intento', 'consumado', 'intento',
        'consumado', 'intento', 'consumado', 'intento', 'consumado'
    ]
})

print(f"Dataset creado: {len(df_with_pii)} registros")
print(f"\n⚠️ Columnas con PII:")
print("  - nombre_completo")
print("  - dni")
print("  - telefono")
print("  - direccion (algunas con dirección exacta)")

df_with_pii.head()

Dataset creado: 15 registros

⚠️ Columnas con PII:
  - nombre_completo
  - dni
  - telefono
  - direccion (algunas con dirección exacta)


/var/folders/b4/vn5cjqrn391dh43wp7w81j2r0000gn/T/ipykernel_43907/1352713272.py:11: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  'fecha_nacimiento': pd.date_range('1960-01-01', periods=15, freq='2Y'),


,id_registro,nombre_completo,fecha_nacimiento,fecha_evento,edad,sexo,dni,telefono,direccion,metodo,municipio,tipo_evento
0,1,Juan García López,1960-12-31,2024-01-07,25,M,12345678A,666123456,"Calle Mayor 123, Madrid",ahorcamiento,Madrid,consumado
1,2,María Rodríguez Pérez,1962-12-31,2024-01-21,30,F,23456789B,677234567,"Avenida Principal 45, Valencia",intoxicacion,Valencia,intento
2,3,Pedro Martínez Sánchez,1964-12-31,2024-02-04,45,M,34567890C,688345678,"Plaza España 7, Barcelona",arma de fuego,Barcelona,consumado
3,4,Ana López Fernández,1966-12-31,2024-02-18,52,F,45678901D,None,"Calle Real 89, Sevilla",ahorcamiento,Sevilla,intento
4,5,Carlos Sánchez García,1968-12-31,2024-03-03,38,M,56789012E,600456789,Madrid,precipitacion,Madrid,consumado


## Paso 2: Ejecutar Análisis Completo

Esto ejecuta los 6 analizadores en paralelo + anonimización automática

In [3]:
# Ejecutar análisis completo
print("🚀 Ejecutando análisis paralelo...\n")

consolidated_json, df_anonymized, anonymization_report = run_parallel_analysis(
    df_with_pii,
    filename="dataset_con_pii.csv",
    auto_anonymize=True
)

print("\n✅ Análisis completado")

INFO:fase3_evaluator.integration.orchestrator:Iniciando análisis paralelo de 'dataset_con_pii.csv' con 15 registros y 12 columnas
INFO:fase3_evaluator.integration.orchestrator:Ejecutando analizadores en paralelo...
INFO:fase3_evaluator.integration.orchestrator:✓ typology completado
INFO:fase3_evaluator.integration.orchestrator:✓ geospatial completado
INFO:fase3_evaluator.integration.orchestrator:✓ semantic completado
INFO:fase3_evaluator.integration.orchestrator:✓ completeness completado
/Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia/cuidar_ia_fase3 4/notebooks/../fase3_evaluator/analyzers/anonymization.py:239: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  detailed_addresses = series[series.str.contains(detailed_pattern, na=False)]
INFO:fase3_evaluator.integration.orchestrator:✓ anonymization completado
INFO:fase3_evaluator.integration.orchestrator:✓ ml_readiness completa

🚀 Ejecutando análisis paralelo...


✅ Análisis completado


## Paso 3: Verificar Anonimización

In [5]:
# Mostrar reporte de anonimización
if anonymization_report:
    print("=" * 60)
    print("REPORTE DE ANONIMIZACIÓN")
    print("=" * 60)
    print(f"Columnas originales: {anonymization_report['original_columns']}")
    print(f"Columnas anonimizadas: {anonymization_report['anonymized_columns']}")
    print(f"Columnas eliminadas: {anonymization_report['columns_removed']}")
    print(f"Transformaciones aplicadas: {anonymization_report['transformations_applied']}")
    
    print("\n📋 Detalles de transformaciones:")
    for col, transform in anonymization_report['transformation_details'].items():
        print(f"  - {col}: {transform}")
    
    print("\n✅ Validación:")
    validation = anonymization_report['validation']
    print(f"  - Seguro: {'Sí' if validation['is_safe'] else 'No'}")
    print(f"  - Riesgo residual PII: {validation['residual_pii_risk']:.1f}")
    if validation['warnings']:
        print(f"  - Advertencias: {len(validation['warnings'])}")
        for warning in validation['warnings']:
            print(f"    ⚠️ {warning}")
else:
    print("ℹ️ No se detectó PII - no se requirió anonimización")

REPORTE DE ANONIMIZACIÓN
Columnas originales: 12
Columnas anonimizadas: 12
Columnas eliminadas: 0
Transformaciones aplicadas: 5

📋 Detalles de transformaciones:
  - id_registro: removed
  - nombre_completo: masked
  - dni: removed
  - telefono: masked_partial
  - direccion: aggregated_to_municipality

✅ Validación:
  - Seguro: Sí
  - Riesgo residual PII: 0.0


## Paso 4: Comparar Datos Originales vs Anonimizados

In [6]:
# Comparación lado a lado
print("COMPARACIÓN: ORIGINAL vs ANONIMIZADO\n")

# Seleccionar columnas que fueron anonimizadas
if anonymization_report:
    pii_cols = list(anonymization_report['transformation_details'].keys())
    
    for col in pii_cols[:3]:  # Primeras 3 columnas
        if col in df_with_pii.columns and col in df_anonymized.columns:
            print(f"\n📊 Columna: {col}")
            print("-" * 40)
            comparison = pd.DataFrame({
                'Original': df_with_pii[col].head(5),
                'Anonimizado': df_anonymized[col].head(5)
            })
            print(comparison)

COMPARACIÓN: ORIGINAL vs ANONIMIZADO


📊 Columna: id_registro
----------------------------------------
   Original  Anonimizado
0         1          NaN
1         2          NaN
2         3          NaN
3         4          NaN
4         5          NaN

📊 Columna: nombre_completo
----------------------------------------
                 Original Anonimizado
0       Juan García López   PERSONA_1
1   María Rodríguez Pérez   PERSONA_2
2  Pedro Martínez Sánchez   PERSONA_3
3     Ana López Fernández   PERSONA_4
4   Carlos Sánchez García   PERSONA_5

📊 Columna: dni
----------------------------------------
    Original  Anonimizado
0  12345678A          NaN
1  23456789B          NaN
2  34567890C          NaN
3  45678901D          NaN
4  56789012E          NaN


## Paso 5: Resumen Ejecutivo del Análisis

In [7]:
# Generar resumen ejecutivo
summary = get_analysis_summary(consolidated_json)

print("=" * 60)
print("RESUMEN EJECUTIVO")
print("=" * 60)

print(f"\n📊 Dataset: {summary['dataset']['rows']} registros, {summary['dataset']['columns']} columnas")

print(f"\n🎯 Scores de Calidad:")
for metric, score in summary['scores'].items():
    emoji = "✅" if score >= 70 else "⚠️" if score >= 50 else "❌"
    print(f"  {emoji} {metric.capitalize()}: {score:.1f}/100")

print(f"\n📈 Score General: {summary['overall_score']:.1f}/100")

print(f"\n⚠️ Issues Críticos:")
print(f"  - PII detectada: {'Sí' if summary['critical_issues']['pii_detected'] else 'No'}")
if summary['critical_issues']['pii_detected']:
    print(f"    Nivel de riesgo: {summary['critical_issues']['pii_risk_level']}")
print(f"  - Completitud crítica: {'Sí' if summary['critical_issues']['completitud_critica'] else 'No'}")
print(f"  - Riesgos de leakage: {summary['critical_issues']['leakage_risks']}")

print(f"\n🚀 Capacidades:")
print(f"  - Geocodificable: {'Sí' if summary['capabilities']['geocodable'] else 'No'}")
print(f"  - ML viable: {'Sí' if summary['capabilities']['ml_viable'] else 'No'}")
print(f"  - Clustering factible: {'Sí' if summary['capabilities']['clustering_feasible'] else 'No'}")

RESUMEN EJECUTIVO

📊 Dataset: 15 registros, 12 columnas

🎯 Scores de Calidad:
  ✅ Completitud: 98.3/100
  ✅ Tipologia: 96.7/100
  ✅ Semantica: 100.0/100
  ⚠️ Geoespacial: 50.0/100
  ❌ Ml_viabilidad: 20.0/100

📈 Score General: 73.0/100

⚠️ Issues Críticos:
  - PII detectada: Sí
    Nivel de riesgo: critico
  - Completitud crítica: No
  - Riesgos de leakage: 0

🚀 Capacidades:
  - Geocodificable: Sí
  - ML viable: No
  - Clustering factible: No


## Paso 6: Generar Diagnóstico con LLM

⚠️ **NOTA**: Esto requiere que tengas configurada tu `OPENAI_API_KEY` en el archivo `.env`

In [8]:
# Verificar si la API key está configurada
if not settings.OPENAI_API_KEY:
    print("❌ ERROR: OPENAI_API_KEY no está configurada")
    print("\nPara configurarla:")
    print("1. Copia .env.example a .env")
    print("2. Edita .env y pega tu API key")
    print("3. Reinicia el kernel del notebook")
else:
    print("🤖 Generando diagnóstico con OpenAI...\n")
    
    try:
        diagnosis_result = generate_diagnosis(
            consolidated_json,
            df_anonymized,
            include_sample=True
        )
        
        print("=" * 60)
        print("DIAGNÓSTICO GENERADO POR IA")
        print("=" * 60)
        print(f"\nModelo usado: {diagnosis_result['model_used']}")
        print(f"Tokens utilizados: {diagnosis_result['tokens_used']}")
        print("\n" + "=" * 60 + "\n")
        print(diagnosis_result['diagnosis'])
        
    except Exception as e:
        print(f"❌ Error al generar diagnóstico: {str(e)}")
        print("\nVerifica:")
        print("1. Que tu API key sea válida")
        print("2. Que tengas créditos disponibles en OpenAI")
        print("3. Que tengas conexión a internet")

INFO:fase3_evaluator.agent.interpreter:Generando diagnóstico con OpenAI...


🤖 Generando diagnóstico con OpenAI...



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:fase3_evaluator.agent.interpreter:Diagnóstico generado exitosamente


DIAGNÓSTICO GENERADO POR IA

Modelo usado: gpt-4o
Tokens utilizados: 7264


## 1. DIAGNÓSTICO GENERAL
El dataset analizado contiene 15 registros y 12 columnas, con datos que incluyen información personal y eventos relacionados con el suicidio. La calidad general del dataset es excelente en términos de completitud, con solo un 1.67% de datos faltantes. Sin embargo, se identificaron problemas críticos relacionados con la presencia de información personal identificable (PII), lo que representa un riesgo ético y legal significativo. Además, hay inconsistencias en la tipología de datos, especialmente en la columna "sexo", que afecta la precisión de los análisis cuantitativos.

## 2. ANÁLISIS QUE SÍ SE PUEDEN REALIZAR
✅ **Análisis Descriptivo de Distribuciones**
📊 Aporta información sobre la distribución de variables clave como edad, sexo y método de suicidio.
🎯 Apoya decisiones de prevención al identificar grupos demográficos de mayor riesgo.

✅ **Análisis de Correlaciones Básicas**
📊 Permi

## Paso 7: Guardar Resultados

In [ ]:

import numpy as np

def convert_numpy_types(obj):
    """Convierte tipos NumPy a tipos nativos Python para JSON serialization"""
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32, np.int16, np.int8)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32, np.float16)):
        return float(obj)
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    else:
        return obj

print("Funcion de conversion cargada")

Funcion de conversion cargada


In [11]:
# Paso 7: Guardar Resultados
import json
from datetime import datetime

output_dir = Path('../data/outputs')
output_dir.mkdir(parents=True, exist_ok=True)

# JSON completo - CONVERTIR ANTES DE GUARDAR
json_path = output_dir / 'analisis_completo.json'
consolidated_json_clean = convert_numpy_types(consolidated_json)
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(consolidated_json_clean, f, indent=2, ensure_ascii=False)
print(f"JSON guardado en: {json_path}")

# Datos anonimizados
csv_path = output_dir / 'datos_anonimizados_flujo_completo.csv'
df_anonymized.to_csv(csv_path, index=False)
print(f"CSV guardado en: {csv_path}")

# Diagnóstico (si existe)
if 'diagnosis_result' in locals() and diagnosis_result:
    md_path = output_dir / 'diagnostico_flujo_completo.md'
    with open(md_path, 'w', encoding='utf-8') as f:
        f.write(f"# Diagnostico Automatico\n")
        f.write(f"Generado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(diagnosis_result['diagnosis'])
    print(f"Diagnostico guardado en: {md_path}")

print(f"\nTodos los archivos en: {output_dir.absolute()}")

JSON guardado en: ../data/outputs/analisis_completo.json
CSV guardado en: ../data/outputs/datos_anonimizados_flujo_completo.csv
Diagnostico guardado en: ../data/outputs/diagnostico_flujo_completo.md

Todos los archivos en: /Users/reinerfuentesferrada/ONLINE_DS_THEBRIDGE_Rei/Proyecto ML/cuidar-ia/cuidar_ia_fase3 4/notebooks/../data/outputs


## ¡Flujo Completado!

Has ejecutado exitosamente:
- ✅ Análisis paralelo de 6 dimensiones
- ✅ Detección automática de PII
- ✅ Anonimización automática
- ✅ Generación de diagnóstico con IA
- ✅ Exportación de resultados

**Próximos pasos**:
1. Prueba con tus propios datos reales
2. Ajusta los umbrales en `config/settings.py`
3. Personaliza los prompts del agente
4. Integra con tu flujo de Streamlit